In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
train = train.rename(columns={'3_labels':'label','comment':'text'})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
test = test.rename(columns={'3_labels':'label','comment':'text'})
test.fillna(0, inplace=True)
print(len(test))



269


In [3]:
train_change_feautre = train.iloc[:, 4:]
test_change_feautre = test.iloc[:, 4:]

train_code_change = train_change_feautre.values
test_code_change = test_change_feautre.values

In [4]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [5]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [6]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [7]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [8]:
len(train)

1524

In [9]:
train['label'].value_counts()

label
Corrective    513
Perfective    510
Adaptive      501
Name: count, dtype: int64

In [10]:
test['label'].value_counts()

label
Corrective    90
Perfective    90
Adaptive      89
Name: count, dtype: int64

In [11]:
len(train_dataset)

1548

In [12]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [13]:
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [14]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
1510,da05ac3adba1223fadc8f898559a5ebd045f6f3e,Vala,gtk+-2.0: set default values for Gtk.Box.pack_...,Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
613,079039e5531a152388776c637884c891974008e5,restlet-framework-java,Fixed generation of cache directives in the- c...,Corrective,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
354,d074c686b3ba0ba6843d0a25dca5ac991f919e68,hadoop,HADOOP-6139. Fix the FsShell help messages for...,Corrective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
794,69438cc0d3444c064f3555ef465176ef5c28f69e,orientdb,Fixed issue on missed saving of the configurat...,Corrective,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
218,a342029d36fc8cdd6be3ab540cd2527ddd022188,elasticsearch,Histogram Facet: Allow to define a key field a...,Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,68302d9b31f48a7ae68109a651054bbd64546956,Mylyn Reviews,Refactoring\n\n- splitted several classes in s...,Perfective,0,0,1,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1574,16cb7d8d59add7b3ef513bba0de6a4fc07e3bc52,aeshell$aesh,"search is more complete, support forward/rever...",Adaptive,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
952,4cbe2ae00aaec2ca4074b06f389f5e50ca235da3,spring-framework,[SPR-8387] Introduced- supports(MergedContextC...,Adaptive,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
508,05d93592ba53123c2483b146d9aab9cc6e112ae1,drools,[DROOLS-37] fix jitting of comparison contrain...,Corrective,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
model_id = "all-roberta-large-v1"

In [16]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found in /all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [17]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    train_code_change = train_change_feautre,
    test_code_change = test_change_feautre,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [18]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:02<00:00,  7.81it/s]
***** Running training *****
  Num examples = 61920
  Num epochs = 1
  Total optimization steps = 3870
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [30:03<00:00, 1803.27s/it]
***** Running evaluation *****


CPU times: user 18min 40s, sys: 13min 32s, total: 32min 13s
Wall time: 30min 10s


{'accuracy': 0.8215613382899628,
 'recall': 0.8215613382899628,
 'f1': 0.8215722116240773}